In [7]:
# Import libraries.
using Turing, StatsPlots, Random

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1278


In [109]:
# Declare our Turing model.
@model function agent(opponent_agent, my_action, opponent_action, opponent_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    if depth > 0 
        opp_action_chain = sample(opponent_agent(agent, opponent_action, my_action, depth-1, opponent_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
        opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
        opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
        opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
        alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
    else
        # Our prior belief about the probability of RPS.
        alpha ~ Dirichlet(ones(3)/3)
    end
    my_action ~ Categorical(vec(alpha))
    return my_action
end

agent (generic function with 7 methods)

In [110]:
chain = sample(agent(agent, 1.0, 1.0, 1, 1, PG, 10, 10), PG(1), 10 , progress = false);
chain

Chains MCMC chain (10×5×1 Array{Float64,3}):

Log evidence      = -22.13228175116812
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Missing     Float64   Float64 

    alpha[1]    0.0433    0.0949     0.0300   missing   -129.5935    0.9161
    alpha[2]    0.4986    0.3634     0.1149   missing      4.7747    1.1764
    alpha[3]    0.4582    0.4054     0.1282   missing      8.5658    1.0759

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0000    0.0000    0.0000    0.0048    0.2497
    alpha[2]    0.0103    0.1570    0.5487    0.8215    0.8890
    alpha[3]    0.0148    0.1157    0.3946    0.8430    0.9883
